In [998]:
reset -fs

In [999]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [1000]:
template = 'https://www.indeed.com/jobs?q={}&l={}'

In [1001]:
def get_url(position, location):
    """ Generate a url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [1002]:
url = get_url('Machine Learning Engineer', 'new york ny')

# Extract raw html

In [1003]:
response = requests.get(url)

In [1004]:
response

<Response [200]>

In [45]:
response.reason

''

In [1005]:
soup = BeautifulSoup(response.text, 'html.parser')

In [1006]:
cards = soup.find_all('td', 'resultContent')

In [1007]:
len(cards)

15

# Prototype the model with a single record

In [1008]:
card = cards[0]

In [37]:
# title_tag = card.div.h2.span

In [862]:
job_title = soup.find(text='Machine Learning Engineer')
print(job_title)

Machine Learning Engineer


In [16]:
# job_title = title_tag.get('title') # JOB TITLE (1)
# print(job_title)

None


In [183]:
company = card.div.span.a

In [184]:
company = soup.find('a', {'data-tn-element':'companyName'}).text.strip() #COMPANY NAME (2)

In [185]:
print(company)

HarperCollins Publishers


In [186]:
job_location = soup.find('div', 'companyLocation').text.strip() #COMPANY LOCATION (3)

In [187]:
print(job_location)

New York, NY•Remote


In [188]:
company_rating = card.div.span.a

In [189]:
company_rating = soup.find('a', {'data-tn-variant':'cmplinktst2'}).text.strip() #COMPANY RATING (4)
print(company_rating)


3.9


In [190]:
today = datetime.today().strftime('%Y-%m-%d') # TODAY's DATE (5)
print(today)

2022-01-23


In [191]:
salary = card.div.div

In [192]:
try:
    salary = soup.find('div', {'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}).text.strip() # SALARY (6)
except AttributeError:
    salary = ''

print(salary)

Estimated $96.1K – $122K a year


In [193]:
if soup.find('span', {'class':'remote-bullet'}):
    remote = True
else:
    remote = False
#REMOTE (7)
print(remote)

# remote = True
# print(remote)

True


In [194]:
age = soup.find('span', 'date').text.strip() #POST AGE (8)
print(age)

Posted30+ days ago


In [195]:
# urgent = soup.find('div', {'class':'urgentlyHiring'}).text #URGENT (9)
# print(urgent)
try:
    urgent = soup.find('div', {'class':'urgentlyHiring'}).text # EASY APPLY
except AttributeError:
    urgent = ''

print(urgent)

In [196]:
# if soup.find('span', {'class': 'ialbl iaTextBlack'}): #EASY APPLY
#     easy_apply = True
# else:
#     easy_apply = False
# print(easy_apply)
# if soup.find('tr', {'class': 'jobCardShelf'}):
#     easy_apply = True
# else:
#     easy_apply = False

# print(easy_apply)

try:
    easy_apply = soup.find('tr', {'class': 'jobCardShelf'}).text # EASY APPLY
except AttributeError:
    easy_apply = ''

print(easy_apply)


Easily apply


# Generalize the model with a function

In [1009]:
def get_record(card):
    """Extract job data from a single record"""
    job_title = soup.find(text='Machine Learning Engineer') # JOB TITLE
    try:
        company = soup.find('a', {'data-tn-element':'companyName'}).text.strip() #COMPANY NAME
    except AttributeError:
        company = ''
    job_location = soup.find('div', 'companyLocation').text.strip() #COMPANY LOCATION
    try:
        company_rating = soup.find('a', {'data-tn-variant':'cmplinktst2'}).text.strip() #COMPANY RATING
    except AttributeError:
        company_rating = ''
    today = datetime.today().strftime('%Y-%m-%d') # TODAY's DATE
    age = soup.find('span', 'date').text.strip() #POST AGE
    if soup.find('span', {'class':'remote-bullet'}): #REMOTE
        remote = True
    else:
        remote = False
    try:
        salary = soup.find('div', {'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}).text.strip() # SALARY
    except AttributeError:
        salary = ''
    try:
        urgent = soup.find('div', {'class':'urgentlyHiring'}).text.strip() #URGENT
    except AttributeError:
        urgent = 'Not Urgent'
    try:
        easy_apply = soup.find('tr', {'class': 'jobCardShelf'}).text # EASY APPLY
    except AttributeError:
        easy_apply = '' 
    
    record = (job_title, company, job_location, company_rating, salary, remote, urgent, easy_apply, age, today)
    return record
    

In [1010]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [1011]:
records[0]

('Machine Learning Engineer',
 'Capital One',
 'New York, NY 10012 (SoHo area)+126 locations',
 '3.9',
 'From $75,000 a year',
 True,
 'Not Urgent',
 '',
 'Posted2 days ago',
 '2022-01-25')

# Getting the next page

In [1012]:
while True:
    try: 
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label':'Next'}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('td', 'resultContent')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [1013]:
len(records)

950

In [1014]:
import pandas as pd

df = pd.DataFrame(records)

In [1015]:
df.to_csv('ml_17.csv')

# Putting it all together

In [35]:
# import csv
# from datetime import datetime
# import requests
# from bs4 import BeautifulSoup

# def get_url(position, location):
#     """ Generate a url from position and location"""
#     template = 'https://www.indeed.com/jobs?q={}&l={}'
#     url = template.format(position, location)
#     return url

# def get_record(card):
#     """Extract job data from a single record"""
#     job_title = title_tag.get('title') # JOB TITLE
#     company = soup.find('a', {'data-tn-element':'companyName'}).text.strip() #COMPANY NAME
#     job_location = soup.find('div', 'companyLocation').text.strip() #COMPANY LOCATION
#     company_rating = soup.find('a', {'data-tn-variant':'cmplinktst2'}).text.strip() #COMPANY RATING
#     today = datetime.today().strftime('%Y-%m-%d') # TODAY's DATE
#     age = soup.find('span', 'date').text.strip() #POST AGE
#     remote = soup.find('span', {'class':'remote-bullet'}) #REMOTE
#     remote = True
#     try:
#         salary = soup.find('div', {'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}).text.strip() # SALARY
#     except AttributeError:
#         salary = ''
    
#     record = (job_title, company, job_location, company_rating, salary, remote, age, today)
#     return record
    
# def main(position, location):
#     """Run the main program"""
#     records = []
#     url = get_url(position, location)
    
#     # extract the job data
#     while True:
#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'html.parser')
#         cards = soup.find_all('td', 'resultContent')
    
#         for card in cards:
#             record = get_record(card)
#             records.append(record)
            
#         try: 
#             url = 'https://www.indeed.com' + soup.find('a', {'aria-label':'Next'}).get('href')
#         except AttributeError:
#             break
            
#     # save the job data
#     with open('results.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         writer.writerow(['JobTitle', 'Company', 'Location', 'Rating', 'Salary', 'Remote', 'PostDate', 'ExtractDate'])
#         writer.writerows(records)
    

In [36]:
# #run the program
# main('data scientist', 'new york, ny')